In [1]:
import os
import pandas as pd
import networkx as nx
import community

In [2]:
inputpath = '/media/sf_VBox_Shared/CaseLaw/graphs/lido/'
leg_links = pd.read_csv(os.path.join(inputpath, 'leg_to_leg_links_min10.tsv'), sep='\t')
case_leg = pd.read_csv(os.path.join(inputpath, 'hr_case_leg_titles.csv'))

In [85]:
nr_references = case_leg.groupby('title').nunique()['source']
nr_references.sort_values(ascending=False).head(20)

title
Wet op de rechterlijke organisatie, Artikel 81     12732
Wetboek van Strafvordering                          2043
Wetboek van Strafrecht                              1395
Wet op de rechterlijke organisatie, Artikel 80a     1222
Wet waardering onroerende zaken                      708
Opiumwet                                             624
Wetboek van Strafvordering, Artikel 457              614
Wetboek van Strafvordering, Artikel 359              603
Wetboek van Strafvordering, Artikel 552a             567
Wetboek van Strafvordering, Artikel 440              432
Wet op de omzetbelasting 1968                        413
Burgerlijk Wetboek Boek 6                            410
Wetboek van Burgerlijke Rechtsvordering              402
Algemene wet inzake rijksbelastingen                 400
Wetboek van Strafvordering, Artikel 365a             397
Wetboek van Strafvordering, Artikel 359a             338
Algemene wet bestuursrecht, Artikel 8:41             337
Algemene wet bestuursrech

In [86]:
leg_links.head()

,cnt,source,target
0,13,Algemene Ouderdomswet,"Algemene wet bestuursrecht, Artikel 8:41"
1,21,Algemene Ouderdomswet,"Wet op de rechterlijke organisatie, Artikel 80a"
2,15,Algemene Ouderdomswet,"Algemene Ouderdomswet, Artikel 53"
3,13,Algemene Ouderdomswet,Algemene nabestaandenwet
4,39,Algemene Ouderdomswet,"Wet op de rechterlijke organisatie, Artikel 81"


## Community detection

In [87]:
graph = nx.Graph()
graph.add_edges_from([(l[2], l[3], {'cnt': l[1]}) for l in leg_links.itertuples()])

In [88]:
# Also make a graph withouth the most common nodes
nodes_to_remove = ["Wet op de rechterlijke organisatie, Artikel 81", 
                   "Wet op de rechterlijke organisatie, Artikel 80a",
                   "Wetboek van Strafvordering",
                   "Wetboek van Strafrecht",
                   "Burgerlijk Wetboek Boek 1",
                   "Burgerlijk Wetboek Boek 2",
                   "Burgerlijk Wetboek Boek 3",
                  "Burgerlijk Wetboek Boek 6",
                   "Burgerlijk Wetboek Boek 7",
                  "Algemene wet bestuursrecht",
                  "Opiumwet",
                  "Wetboek van Burgerlijke Rechtsvordering", 
                   "Faillissementswet"]

In [89]:
graph2 = graph.copy()
for n in nodes_to_remove:
    graph2.remove_node(n)

In [90]:
commmunities_weighted = community.best_partition(graph, weight='cnt')
commmunities_unweighted = community.best_partition(graph)
commmunities_weighted_small = community.best_partition(graph, weight='cnt', resolution=0.5)
commmunities_unweighted_small = community.best_partition(graph, resolution=0.5)

In [91]:
communities_weighted2 = community.best_partition(graph2, weight='cnt')

In [92]:
leg_nodes_df = pd.DataFrame()
leg_nodes_df['louvain_weighted'] = pd.Series(commmunities_weighted)
leg_nodes_df['louvain_unweighted'] = pd.Series(commmunities_unweighted)
leg_nodes_df['louvain_weighted_small'] = pd.Series(commmunities_weighted_small)
leg_nodes_df['louvain_unweighted_small'] = pd.Series(commmunities_unweighted_small)
leg_nodes_df['louvain_weighted_sub'] = pd.Series(communities_weighted2)
leg_nodes_df = leg_nodes_df.reset_index().rename(columns={"index": "name"})

In [93]:
leg_nodes_df[leg_nodes_df['name']=="Wetboek van Strafrecht"]

,name,louvain_weighted,louvain_unweighted,louvain_weighted_small,louvain_unweighted_small,louvain_weighted_sub
405,Wetboek van Strafrecht,1,5,1,4,NaN


## meta info legislation

In [94]:
leg_nodes_df = leg_nodes_df.set_index(['name'])
leg_nodes_df['nr_references'] = nr_references
leg_nodes_df = leg_nodes_df.reset_index()
leg_nodes_df.head()

,name,louvain_weighted,louvain_unweighted,louvain_weighted_small,louvain_unweighted_small,louvain_weighted_sub,nr_references
0,Algemene Ouderdomswet,3,4,3,1,1.0,111
1,"Algemene Ouderdomswet, Artikel 53",3,4,3,1,1.0,15
2,"Algemene Ouderdomswet, Artikel 6",3,4,3,1,1.0,14
3,Algemene bijstandswet,3,3,3,1,3.0,59
4,Algemene nabestaandenwet,3,4,3,1,1.0,64


In [95]:
leg_nodes_df.to_csv(os.path.join(inputpath, 'leg_to_leg_nodes_min10.tsv'), index=False, sep='\t')
leg_nodes_df.to_csv(os.path.join(inputpath, 'leg_to_leg_nodes_min10.csv'), index=False)

In [96]:
leg_nodes_df['book'] = leg_nodes_df['name'].str.split(',').map(lambda l: l[0])

In [97]:
leg_nodes_df['book'].value_counts()

Wetboek van Strafvordering                                                                   137
Wetboek van Strafrecht                                                                        85
Wetboek van Burgerlijke Rechtsvordering                                                       41
Algemene wet bestuursrecht                                                                    30
Algemene wet inzake rijksbelastingen                                                          22
Burgerlijk Wetboek Boek 6                                                                     21
Faillissementswet                                                                             17
Wet op de omzetbelasting 1968                                                                 17
Burgerlijk Wetboek Boek 7                                                                     15
Wet bijzondere opnemingen in psychiatrische ziekenhuizen                                      15
Burgerlijk Wetboek Boek 1     

## look into clusters

In [98]:
case_to_leg_merged = case_leg.merge(leg_nodes_df, left_on='title', right_on='name')
case_to_leg_merged.head()

,source,title,name,louvain_weighted,louvain_unweighted,louvain_weighted_small,louvain_unweighted_small,louvain_weighted_sub,nr_references,book
0,http://linkeddata.overheid.nl/terms/jurisprude...,Wet inkomstenbelasting 2001,Wet inkomstenbelasting 2001,19,3,31,28,51.0,213,Wet inkomstenbelasting 2001
1,http://linkeddata.overheid.nl/terms/jurisprude...,Wet inkomstenbelasting 2001,Wet inkomstenbelasting 2001,19,3,31,28,51.0,213,Wet inkomstenbelasting 2001
2,http://linkeddata.overheid.nl/terms/jurisprude...,Wet inkomstenbelasting 2001,Wet inkomstenbelasting 2001,19,3,31,28,51.0,213,Wet inkomstenbelasting 2001
3,http://linkeddata.overheid.nl/terms/jurisprude...,Wet inkomstenbelasting 2001,Wet inkomstenbelasting 2001,19,3,31,28,51.0,213,Wet inkomstenbelasting 2001
4,http://linkeddata.overheid.nl/terms/jurisprude...,Wet inkomstenbelasting 2001,Wet inkomstenbelasting 2001,19,3,31,28,51.0,213,Wet inkomstenbelasting 2001


In [101]:
com_name = 'louvain_weighted_sub'

grouped_by_com = leg_nodes_df.groupby(com_name)
community_summary = pd.DataFrame()
community_summary['nodes'] = grouped_by_com['name'].apply(lambda l: "|".join(list(sorted(l))))
community_summary['nr_leg_nodes'] = grouped_by_com['name'].nunique()
community_summary['nr_cases'] = case_to_leg_merged.groupby(com_name)['source'].nunique()
community_summary = community_summary.sort_values('nr_cases', ascending=False)
community_summary.head(50)

,nodes,nr_leg_nodes,nr_cases
louvain_weighted_sub,,,
8.0,"Algemene wet inzake rijksbelastingen, Artikel ...",52,2749
1.0,"Algemene Ouderdomswet|Algemene Ouderdomswet, A...",22,1675
16.0,Algemene wet inzake rijksbelastingen|Algemene ...,26,1210
5.0,"Burgerlijk Wetboek Boek 3, Artikel 86|Wet op h...",31,993
26.0,"Wetboek van Strafvordering, Artikel 278|Wetboe...",15,873
35.0,"Wegenverkeerswet 1994, Artikel 67|Wet aansprak...",14,783
24.0,"Opiumwet, Artikel 1|Opiumwet, Artikel 10|Opium...",14,731
2.0,Wet op belastingen van rechtsverkeer|Wet op be...,22,606
28.0,"Politiewet 1993|Politiewet 1993, Artikel 2|Weg...",20,605


In [102]:
community_summary.to_csv(os.path.join(inputpath, 'leg_to_leg_communities.csv'))